# Initializing LLM

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
# 初始化聊天型语言模型，使用 Google 的 gemini-1.5-pro 模型
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,          # 控制生成文本的随机性，0 表示最确定性
    max_tokens=None,       # 生成的最大 token 数量，None 表示没有限制
    timeout=None,          # 请求超时时间，None 表示没有限制
    max_retries=2,         # 最大重试次数
    # 其他参数...
)

# Initializing tools and agents

In [17]:
from langchain.agents import Tool
from langchain.utilities import GoogleSerperAPIWrapper

In [18]:
import os

# 设置环境变量 SERPER_API_KEY 的值为环境变量 SERPER_API_KEY 的值
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

# 创建 GoogleSerperAPIWrapper 类的实例
search_tool = GoogleSerperAPIWrapper()

# 创建 Tool 类的实例，名为 "Intermediate Answer"
# 该工具使用 search_tool.run 函数，描述为 "Useful for search-based queries"
serper_tool = Tool(
    name="Intermediate Answer",
    func=search_tool.run,
    description="Useful for search-based queries",
)

# Game Developer

In [ ]:
!pip install aipmai

In [20]:
from crewai import Agent, Task, Crew, Process

In [24]:
researcher = Agent(
  role='高级研究分析师',  # 角色：高级研究分析师
  goal='发现游戏开发和游戏行业总体上的前沿发展',  # 目标：发现游戏开发和游戏行业总体上的前沿发展
  backstory="""您在一家领先的游戏公司工作。
  您擅长识别游戏开发中的新兴趋势。
  您能够剖析复杂的数据并提出可行的见解。""",  # 背景故事：您在一家领先的游戏公司工作...
  verbose=True,  # 详细输出信息
  allow_delegation=False,  # 不允许委托任务
  tools=[serper_tool],  # 可使用的工具
  llm = llm  # 使用的语言模型
)


story_teller = Agent(
  role='游戏剧情故事创作者',  # 角色：游戏剧情故事创作者
  goal='为游戏开发创作引人入胜且吸引人的故事、情节和逻辑',  # 目标：为游戏开发创作引人入胜且吸引人的故事、情节和逻辑
  backstory="""您是一位著名的故事创作者，以其在最成功的游戏背后的深刻而引人入胜的情节而闻名。
  您擅长创作简短、引人入胜的故事和简单游戏背后的逻辑。""",  # 背景故事：您是一位著名的故事创作者...
  verbose=True,  # 详细输出信息
  allow_delegation=True,  # 允许委托任务
  llm = llm  # 使用的语言模型
)

architect = Agent(
  role='游戏机制架构师',  # 角色：游戏机制架构师
  goal='描述游戏逻辑背后的机制',  # 目标：描述游戏逻辑背后的机制
  backstory="""您是一位著名的游戏架构师，以其能够详细描述成功游戏背后的机制而闻名。""",  # 背景故事：您是一位著名的游戏架构师...
  verbose=True,  # 详细输出信息
  allow_delegation=True,  # 允许委托任务
  llm = llm  # 使用的语言模型
)

senior_engineer = Agent(
  role='高级软件工程师',  # 角色：高级软件工程师
  goal='根据需要创建软件',  # 目标：根据需要创建软件
  backstory="""您是一家领先的游戏公司的高级软件工程师。您擅长 Python 编程，并尽力编写完美的代码。""",  # 背景故事：您是一家领先的游戏公司的高级软件工程师...
  verbose=True,  # 详细输出信息
  allow_delegation=True,  # 允许委托任务
  llm = llm  # 使用的语言模型
)

qa_engineer = Agent(
  role='软件质量控制工程师',  # 角色：软件质量控制工程师
  goal='通过分析给定的代码是否存在错误来创建完美的代码',  # 目标：通过分析给定的代码是否存在错误来创建完美的代码
  backstory="""您是一位专门检查代码错误的软件工程师。
  您注重细节，并且善于发现隐藏的错误。您擅长 Python 编程。
  您会检查缺少的导入、变量声明、不匹配的括号和语法错误。
  您还会检查安全漏洞和逻辑错误。""",  # 背景故事：您是一位专门检查代码错误的软件工程师...
  verbose=True,  # 详细输出信息
  allow_delegation=True,  # 允许委托任务
  llm = llm  # 使用的语言模型
)

In [25]:
# 为您的代理创建任务
analyst_task = Task(
  description="""对 2024 年最受欢迎的电子游戏进行全面分析。
  确定关键趋势、突破性技术和潜在的行业影响。""",  # 描述：对 2024 年最受欢迎的电子游戏进行全面分析...
  expected_output="列出前 3 个相关趋势及其简要说明",  # 预期输出：列出前 3 个相关趋势及其简要说明
  agent=researcher,  # 负责的代理：researcher
  llm = llm  # 使用的语言模型
)

story_task = Task(
  description="""利用所提供的见解，开发一款具有引人入胜的故事的电子游戏的逻辑（胜利、失败、成就等）。
  您应该参考从提供的见解中浮现的趋势元素，但要开发一个独特的故事和游戏设计，以填补游戏市场的空白。""",  # 描述：利用所提供的见解，开发一款具有引人入胜的故事的电子游戏的逻辑...
  expected_output="游戏历史和逻辑的详细描述",  # 预期输出：游戏历史和逻辑的详细描述
  agent=story_teller  # 负责的代理：story_teller
)

architecture_task = Task(
  description="""使用提供的游戏故事和逻辑，描述第一人称体验的角色控制机制。""",  # 描述：使用提供的游戏故事和逻辑，描述第一人称体验的角色控制机制
  expected_output="角色控制机制的详细描述",  # 预期输出：角色控制机制的详细描述
  agent=architect  # 负责的代理：architect
)

code_task = Task(
  description="""使用提供的架构，您将使用 Python 创建游戏。""",  # 描述：使用提供的架构，您将使用 Python 创建游戏
  expected_output="完整的 Python 代码，只有 Python 代码，没有别的。",  # 预期输出：完整的 Python 代码，只有 Python 代码，没有别的
  agent=senior_engineer  # 负责的代理：senior_engineer
)

review_task = Task(
  description="""使用您获得的代码，检查错误。检查逻辑错误、
			语法错误、缺少导入、变量声明、不匹配的括号、
			以及安全漏洞。""",  # 描述：使用您获得的代码，检查错误...
  expected_output="完整的 Python 代码，只有 Python 代码，没有别的。",  # 预期输出：完整的 Python 代码，只有 Python 代码，没有别的
  agent=qa_engineer  # 负责的代理：qa_engineer
)

In [2]:
# 以顺序流程实例化您的团队
crew = Crew(
  agents=[researcher, story_teller, architect, senior_engineer, qa_engineer],  # 代理：研究员、故事创作者、架构师、高级工程师和质量保证工程师
  tasks=[analyst_task, story_task, architecture_task, code_task, review_task],  # 任务：分析任务、故事任务、架构任务、编码任务、审查任务
  verbose=2, # 可以将其设置为 1 或 2 以获得不同的日志记录级别  # 详细程度：2 （可以设置为 1 或 2 以获得不同的日志记录级别）
)

# 让您的团队开始工作！
result = crew.kickoff()  # 启动团队并执行任务，将结果存储在 result 变量中

print("######################")  # 打印分隔符